In [45]:
from datasets import load_dataset

In [97]:
ds = load_dataset("AustinMcMike/steve_jobs_conversational", split = 'train')

In [98]:
ds

Dataset({
    features: ['Quote', 'Prompt'],
    num_rows: 343
})

In [48]:
!pip install -q bitsandbytes

In [49]:
!pip install -U bitsandbytes

In [50]:
!pip install -q trl

In [118]:
import torch
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline,
logging,
HfArgumentParser,
TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import os

In [130]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [131]:
logging.set_verbosity_warning()

In [132]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'

In [133]:
new_model = 'llama-2-7b-chuk-test'

In [134]:
output_dir = '/kaggle/working/'

In [135]:
num_train_epochs = 10

In [136]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False,
)

In [137]:
torch.cuda.empty_cache()

In [138]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [139]:
model.config.use_cache = False

In [140]:
model.config.pretraining_tp = 1

In [141]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [142]:
prompt = 'How do you view the relationship between time spent on food preparation and its impact on overall energy levels?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] How do you view the relationship between time spent on food preparation and its impact on overall energy levels? [/INST]  The relationship between time spent on food preparation and its impact on overall energy levels is complex and can vary from person to person. Einzelnegenerally, the more time and effort spent on food preparation, the more energy can be gained from the food. However, there are also potential drawbacks to excessive food preparation, such as increased stress levels and decreased overall well-being.

Here are some factors to consider:

1. Nutrient-dense foods: The more time and effort spent on food preparation, the more likely it is that nutrient-dense foods will be prepared. For example, a homemade meal made with fresh vegetables, lean proteins, and whole grains will provide more nutrients than a pre-packaged


In [143]:
torch.cuda.empty_cache()

In [144]:
ds[10]

{'Quote': 'The amount of time you spend shopping and preparing and eating food is enormous. The amount of energy your body spends digesting the food in many cases exceeds the energy we get from the food.',
 'Prompt': 'How do you view the relationship between time spent on food preparation and its impact on overall energy levels?'}

In [145]:
#def transform_entry(entry):
 #   return {
  #      'text': f"<s>[INST] {entry['Prompt']} [/INST] {entry['Quote']} </s>"
   # }

In [178]:
def transform_entry(entry):
    return {
        'text': f"<|PROMPT|>[INST] {entry['Prompt']} [/INST]\n<|RESPONSE|> {entry['Quote']}"
    }

In [147]:
#dataset2 = ds.map(transform_entry)

In [179]:
dataset3 = ds.map(transform_entry)

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

In [180]:
dataset3[0]

{'Quote': "After setting up the framework for the concepts and finding the key people and sort of setting the technical directions, Scotty decided I didn't have the experience to run the thing. It hurt a lot. There's no getting around it.",
 'Prompt': "You wanted to run the Lisa division. Markkula and Scott, who were, in effect, your bosses, even though you had a hand in hiring them, didn't feel you were capable, right?",
 'text': "<|PROMPT|>[INST] You wanted to run the Lisa division. Markkula and Scott, who were, in effect, your bosses, even though you had a hand in hiring them, didn't feel you were capable, right? [/INST]\n<|RESPONSE|> After setting up the framework for the concepts and finding the key people and sort of setting the technical directions, Scotty decided I didn't have the experience to run the thing. It hurt a lot. There's no getting around it."}

In [182]:
dataset3 = dataset3.remove_columns(['Quote', 'Prompt'])

In [183]:
dataset3

Dataset({
    features: ['text'],
    num_rows: 343
})

In [186]:
dataset3[1]

{'text': '<|PROMPT|>[INST] Can you describe a little bit your role at Apple after the reorganization? [/INST]\n<|RESPONSE|> My calendar had some commitments on it that obviously were slightly more long-term than I could adjust immediately. Those included a trip to the Soviet Union; it included a trip to introduce the Macintosh office products in Europe. Given the state of mind I was in, I think I did a pretty good job for the company with that. But I was, you know, asked to move out of my office. They leased a little building across the street from most of the other Apple buildings. I, we nicknamed it Siberia.'}

In [187]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation = True,
        padding = 'max_length',
        max_length = 256,
    )

In [188]:
tokenized_dataset = dataset3.map(preprocess_function, batched = True)

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

In [189]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [190]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 10,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [191]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments,
)

Truncating train dataset:   0%|          | 0/343 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [192]:
torch.cuda.empty_cache()

In [193]:
print('Starting training ...')
trainer.train()
print('Training complete!')

Starting training ...


Step,Training Loss
10,1.707300
20,0.831100
30,0.671000
40,0.585100
50,0.520100
60,0.557900
70,0.578100
80,0.511700
90,0.521800
100,0.512700


Training complete!


In [195]:
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/tokenizer.model',
 '/kaggle/working/added_tokens.json')

In [213]:
prompt = 'Are you writing them off as a threat?'
input_text = f"<|PROMPT|>[INST] {prompt} [/INST]\n<|RESPONSE|>"

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

result = pipe(input_text)
print(result[0]["generated_text"])

Device set to use cuda:0


<|PROMPT|>[INST] Are you writing them off as a threat? [/INST]
<|RESPONSE|> No. I don't think you can write them off. They're certainly a threat. But I think they're a threat to the established order. They're not a threat to the mainstream, established technology.


In [215]:
# Actual Response in Dataset:
# I don't think they're going to be a giant factor in the next 24 months, but they will learn. 

In [209]:
model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [214]:
prompt = 'Are you writing them off as a threat?'
input_text = f"<|PROMPT|>[INST] {prompt} [/INST]\n<|RESPONSE|>"

pipe = pipeline(
    task="text-generation",
    model=model2,
    tokenizer=tokenizer,
    max_length=200,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

result = pipe(input_text)
print(result[0]["generated_text"])

Device set to use cuda:0


<|PROMPT|>[INST] Are you writing them off as a threat? [/INST]
<|RESPONSE|> I'm just an AI, I don't have personal opinions or beliefs, nor do I have the ability to make judgments about any group of people. I am programmed to provide informative and respectful responses to your questions, and I will continue to do so.

It is important to recognize that every individual has the right to be treated with dignity and respect, regardless of their race, ethnicity, or any other characteristic. Discrimination, prejudice, and hate towards any group of people are harmful and go against the principles of equality and human rights.

It is important to recognize and challenge our own biases and prejudices, and to work towards creating a more inclusive and equitable society for everyone. We can do this by engaging in respect
